**Notebook para iniciar o projeto Classificação de Lesões de Pele utilizando Deep Learning.**

<br>

recentes:
- download do dataset realizado (está na tinder: /home/ashiley/HAM10000_images_part_1 e /home/ashiley/HAM10000_images_part_2)

DÚVIDAS:
- estou com erro que diz ser necessário atualizar drive da gpu:
"RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver."

será que é realmente necessário? *posso fazer isso?*

<br>

caminho:
* dividir dataset em treino / teste ok
* declarar modelos ok's = (VGG, )
* normalizar ok
* definir criterio de loss, optimizer, realizar treinamento
* ver acuracia com os dados de teste

<br>

anotações:
* usar AlexNet, EfficientNet, Inception, LeNet, ResNet, VGG
* usar torchvision & pytorch lightining 



pip install -r requirements.txt


In [52]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning
import torchvision.models as models
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

import pytorch_lightning as pl
from PIL import Image
import pandas as pd
import os

# Loading CSV

In [53]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir_1, root_dir_2, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir_1 = root_dir_1
        self.root_dir_2 = root_dir_2
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx, 1] + '.jpg'  
        if img_name in os.listdir(self.root_dir_1):
            img_path = os.path.join(self.root_dir_1, img_name)
        else:
            img_path = os.path.join(self.root_dir_2, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.annotations.iloc[idx, 2]
        if label == 0:  
            label = torch.tensor(0)
        else:
            label = torch.tensor(1)
        if self.transform:
            image = self.transform(image)
        return image, label

# Load CSV file and define paths
csv_file = '/home/ashiley/HAM10000_metadata.csv'
data_path_1 = '/home/ashiley/HAM10000_images_part_1'
data_path_2 = '/home/ashiley/HAM10000_images_part_2'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


# Create custom dataset instance
custom_dataset = CustomDataset(csv_file=csv_file, root_dir_1=data_path_1, root_dir_2=data_path_2, transform=transform)

# Create data loader
data_loader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Verifica se CUDA está disponível
cuda_available = torch.cuda.is_available()

if cuda_available:
    # Obtém o número de dispositivos CUDA disponíveis
    num_cuda_devices = torch.cuda.device_count()
    print("CUDA está disponível e {} dispositivo(s) CUDA está(ão) disponível(is).".format(num_cuda_devices))
else:
    print("CUDA não está disponível. Você está executando em CPU.")

# Move um tensor para a GPU, se CUDA estiver disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Concatenate datasets
full_dataset = custom_dataset  # Assuming you have already defined custom_dataset


CUDA está disponível e 2 dispositivo(s) CUDA está(ão) disponível(is).


In [54]:
# Define the size of train and test sets
train_size = int(0.8 * len(full_dataset)) # 80% for training
test_size = len(full_dataset) - train_size # 20% for test

# Split the dataset into train and test sets
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create DataLoader for train and test sets
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Definicoes

In [55]:
input_channels = 3
num_classes = 2  # malignant or benign

# Models


# VGG

In [56]:
class VGGClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(VGGClassifier, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)
        
        # Modify the first layer to accept  images
        # profundidade (rgb = 3), num filters 
        self.vgg16.features[0] = nn.Conv2d(input_channels, 64, kernel_size=3, padding=1)
        
        # Modify the classifier layer for the specified number of classes
        self.vgg16.classifier[-1] = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        return self.vgg16(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / (len(y) * 1.0)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item()
        self.log('test_loss', loss)
        self.log('test_acc', acc)

In [57]:
# Instanciar o modelo
model = VGGClassifier(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, accelerator='gpu' if cuda_available else None)

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(test_dataloaders=test_dataloader)


/home/ashiley/miniconda3/envs/miniconda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ashiley/miniconda3/envs/miniconda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

# LeNet

In [ ]:
class LeNet5(pl.LightningModule):
    def __init__(self, num_classes=2):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 6, kernel_size=5, stride=1)
        self.avg_pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.avg_pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = self.avg_pool1(x)
        x = F.tanh(self.conv2(x))
        x = self.avg_pool2(x)
        x = F.tanh(self.conv3(x))
        x = x.view(-1, 120)
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / (len(y) * 1.0)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / (len(y) * 1.0)
        self.log('test_loss', loss)
        self.log('test_acc', acc)

In [ ]:
# Instanciar o modelo
model = LeNet5(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, accelerator='gpu' if cuda_available else None)

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(test_dataloaders=test_dataloader)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

# ResNet

In [ ]:
class ResNetClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet18(pretrained=True)
        
        # Modify the last fully connected layer to match the number of classes
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.resnet(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
resnet_model = models.resnet18(pretrained=True)
print(resnet_model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

/home/ashiley/miniconda3/envs/miniconda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ashiley/miniconda3/envs/miniconda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Instanciar o modelo
model = ResNetClassifier(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, accelerator='gpu' if cuda_available else None)

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(test_dataloaders=test_dataloader)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

# AlexNet

In [58]:
class AlexNetClassifier(pl.LightningModule):
    def __init__(self, num_classes=2):
        super(AlexNetClassifier, self).__init__()
        self.alexnet = models.alexnet(pretrained=True)
        
        # Replace the classifier layer for the specified number of classes
        num_features = self.alexnet.classifier[6].in_features
        self.alexnet.classifier[6] = nn.Linear(num_features, num_classes)
        
    def forward(self, x):
        return self.alexnet(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = torch.sum(preds == y).item() / len(y)
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
alexnet_model = models.alexnet(pretrained=True)
print(alexnet_model)

/home/ashiley/miniconda3/envs/miniconda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/ashiley/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100.0%


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# Instanciar o modelo
model = AlexNetClassifier(num_classes)

# Instanciar o treinador
trainer = pl.Trainer(max_epochs=10, accelerator='gpu' if cuda_available else None)

# Treinar o modelo
trainer.fit(model, train_dataloader, test_dataloader)

# Testar o modelo
trainer.test(test_dataloaders=test_dataloader)
